# Chapter 12: A Language model from scratch

In [ ]:
import requests
import gzip
import pandas as pd

# URL of the gzipped text file
url = "https://github.com/lsb/human-numbers/blob/trunk/one-hundred-thousand-numbers.txt.gz?raw=true"

# Downloading the file using requests
response = requests.get(url)
response.raise_for_status()  # This will raise an error if the download failed

# Unzipping the content
content = gzip.decompress(response.content).decode('utf-8')

# Since the file contains numbers, each number on a new line, we can split the content into a list
numbers = content.splitlines()

In [ ]:
text = ' '.join(x for x in numbers)
tokens = text.split(' ')

tokens[:10]

In [ ]:
vocab = sorted(list(set(tokens)))
len(vocab)

In [ ]:
# token to numbers
word2idx = {w:i for i, w in enumerate(vocab)}
nums = [word2idx[i] for i in tokens]
nums[:10]

In [ ]:
for idx in nums[:25]:
    print(idx, vocab[idx])

### dataset prep

In [ ]:
dummy_tokens = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']  # Example list

for i in range(0, len(dummy_tokens) - 3, 1):
    three_tokens = dummy_tokens[i:i+3]  # Get a slice of three tokens
    next_token = dummy_tokens[i+3]      # Get the token immediately following the slice
    print(f"Three tokens: {three_tokens}, Next token: {next_token}, Step: {i}")

In [ ]:
### predict next token based on three previous tokens; book uses step size of 3 which has no overlap, i prefer 1
[(tokens[i:i+3], tokens[i+3]) for i in range(0, len(tokens)-3, 1)][2000:2020] # change from 4-2 to 3-1

In [ ]:
import torch

# Assuming 'mps_device' is defined as your MPS device
mps_device = torch.device('mps')

In [ ]:
# create dataset karpathy style
xs = []
ys = []
for i in range(0, len(tokens) - 3, 1):
    three_tokens = torch.tensor(nums[i:i+3])  # Get a slice of three tokens
    next_token = torch.tensor(nums[i+3])      # Get the token immediately following the slice
    xs.append(three_tokens)
    ys.append(next_token)

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

class LLMModel1(nn.Module):
    def __init__(self, vocab_size, n_hidden):
        super(LLMModel1, self).__init__()  # Initialize the superclass
        self.i_h = nn.Embedding(vocab_size, n_hidden) #vocab to hidden
        self.h_h = nn.Linear(n_hidden, n_hidden) # hidden to hidden
        self.h_o = nn.Linear(n_hidden, vocab_size) # hidden to vocab (logits)
        
    def forward(self, x):
        """hidden states are accumulated. subsequent hidden state is added to embedding of next token before being passed through next linear layer and ReLU"""
        # create first hidden state from first word
        # embed --> linear --> relu
        h = F.relu(self.h_h(self.i_h(x[:, 0])))

        # second hidden state from second word
        h = h + self.i_h(x[:, 1])
        h = F.relu(self.h_h(h))

        # hidden state from third word
        h = h + self.i_h(x[:, 2])
        h = F.relu(self.h_h(h))
        return self.h_o(h)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Assuming X and Y are your data tensors
X = torch.stack(xs)
Y = torch.stack(ys)
dataset = TensorDataset(X, Y)

# Calculate the sizes of splits
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Split the dataset (this method shuffles the data)
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Split the dataset without shuffling
train_dataset = TensorDataset(X[:train_size], Y[:train_size])
val_dataset = TensorDataset(X[train_size:], Y[train_size:])

# Create data loaders with drop_last=True to ensure all batches have the same size
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, drop_last=True)


In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, device):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for batch in progress_bar:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update progress bar
            avg_loss = total_loss / total
            accuracy = 100 * correct / total
            progress_bar.set_postfix(loss=avg_loss, accuracy=f'{accuracy:.2f}%')

        train_losses.append(total_loss / len(train_loader))
        train_accuracies.append(100 * correct / total)

        # Validation phase
        model.eval()
        total_loss = 0
        correct = 0
        total = 0
        progress_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Validation]')
        with torch.no_grad():
            for batch in progress_bar:
                inputs, labels = batch
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                # Update progress bar
                avg_loss = total_loss / total
                accuracy = 100 * correct / total
                progress_bar.set_postfix(loss=avg_loss, accuracy=f'{accuracy:.2f}%')

        val_losses.append(total_loss / len(val_loader))
        val_accuracies.append(100 * correct / total)

    return train_losses, val_losses, train_accuracies, val_accuracies


In [ ]:
vocab_size = len(vocab)
n_hidden = 64
model = LLMModel1(vocab_size, n_hidden)
model.to(mps_device)  # Move model to MPS device

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

num_epochs = 2
device = mps_device

train_losses, val_losses, train_accuracies, val_accuracies = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, device)

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(train_losses, val_losses, train_accuracies=None, val_accuracies=None):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(12, 4))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'bo-', label='Training loss')
    plt.plot(epochs, val_losses, 'ro-', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot accuracy if provided
    if train_accuracies and val_accuracies:
        plt.subplot(1, 2, 2)
        plt.plot(epochs, train_accuracies, 'bo-', label='Training accuracy')
        plt.plot(epochs, val_accuracies, 'ro-', label='Validation accuracy')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

    plt.show()


In [ ]:
plot_training_history(train_losses, val_losses, train_accuracies, val_accuracies)

### refactor to RNN

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

class LLMModel2(nn.Module):
    def __init__(self, vocab_size, n_hidden):
        super(LLMModel2, self).__init__()  # Initialize the superclass
        self.i_h = nn.Embedding(vocab_size, n_hidden) #vocab to hidden
        self.h_h = nn.Linear(n_hidden, n_hidden) # hidden to hidden
        self.h_o = nn.Linear(n_hidden, vocab_size) # hidden to vocab (logits)
        
    def forward(self, x):
        """hidden states are accumulated. subsequent hidden state is added to embedding of next token before being passed through next linear layer and ReLU"""
        h = 0
        for i in range(3):
            h = h + self.i_h(x[:, i])
            h = F.relu(self.h_h(h))
        return self.h_o(h)



In [ ]:
torch.mps.empty_cache()

In [ ]:
vocab_size = len(vocab)
n_hidden = 64
model = LLMModel2(vocab_size, n_hidden)
model.to(mps_device)  # Move model to MPS device

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

train_losses, val_losses, train_accuracies, val_accuracies = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, device)

### maintaining hidden state

- instead of resetting to 0 for each forward

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

class LLMModel3(nn.Module):
    def __init__(self, vocab_size, n_hidden):
        super(LLMModel3, self).__init__()  # Initialize the superclass
        self.i_h = nn.Embedding(vocab_size, n_hidden) #vocab to hidden
        self.h_h = nn.Linear(n_hidden, n_hidden) # hidden to hidden
        self.h_o = nn.Linear(n_hidden, vocab_size) # hidden to vocab (logits)
        self.h = 0
        
    def forward(self, x):
        """hidden states are accumulated. subsequent hidden state is added to embedding of next token before being passed through next linear layer and ReLU"""
        for i in range(3):
            self.h = self.h + self.i_h(x[:, i])
            self.h = F.relu(self.h_h(self.h))
        out = self.h_o(self.h)
        self.h = self.h.detach() # stop tracking previous gradients
        return out

    def reset(self):
        # Resets or reinitializes the hidden state to zero
        self.h = 0


# detach means we only calculate gradients of previous 3 steps (for loop) aka the steps used to calculate this self.h; this is used to avoid vanishing/exploding graidnets

In [ ]:
torch.mps.empty_cache()


model = LLMModel3(vocab_size, n_hidden)
model.to(mps_device)  # Move model to MPS device

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

train_losses, val_losses, train_accuracies, val_accuracies = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, device)

slight boost in performance...

### now instead of outputting one word per three input words, output after each step...

this gives it more signal for back prop

In [ ]:
sl = 16
nums[i:i+sl], nums[i+1:i+sl+1]

In [ ]:
# create dataset karpathy style
sl = 16

xs = []
ys = []
for i in range(0, len(nums) - sl-1, 1):
    sixteen_tokens = torch.tensor(nums[i:i+16])  # Get a slice of three tokens
    next_sixteen_tokens = torch.tensor(nums[i+1: i+sl+1])      # Get the token immediately following the slice
    xs.append(sixteen_tokens)
    ys.append(next_sixteen_tokens)

# Assuming X and Y are your data tensors
X = torch.stack(xs)
Y = torch.stack(ys)
dataset = TensorDataset(X, Y)

# Calculate the sizes of splits
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Split the dataset (this method shuffles the data)
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Split the dataset without shuffling
train_dataset = TensorDataset(X[:train_size], Y[:train_size])
val_dataset = TensorDataset(X[train_size:], Y[train_size:])

# Create data loaders with drop_last=True to ensure all batches have the same size
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, drop_last=True)


In [ ]:
i_ = 1
xs[i_], ys[i_]

for num in xs[i_]:
    print(vocab[num])

print('\n')

for num in ys[i_]:
    print(vocab[num])

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

class LLMModel4(nn.Module):
    def __init__(self, vocab_size, n_hidden):
        super(LLMModel4, self).__init__()  # Initialize the superclass
        self.i_h = nn.Embedding(vocab_size, n_hidden) #vocab to hidden
        self.h_h = nn.Linear(n_hidden, n_hidden) # hidden to hidden
        self.h_o = nn.Linear(n_hidden, vocab_size) # hidden to vocab (logits)
        self.h = 0
        
    def forward(self, x):
        """output a prediction after each word

        will return outs as shape batch_size x seq_len x vocab_sz
        """
        outs = [] # (batch_size x vocab_sz)
        for i in range(sl): # CHANGED TO SL
            self.h = self.h + self.i_h(x[:, i])
            self.h = F.relu(self.h_h(self.h))
            outs.append(self.h_o(self.h))
        self.h = self.h.detach() # stop tracking previous gradients
        return torch.stack(outs, dim=1)

    def reset(self):
        # Resets or reinitializes the hidden state to zero
        self.h = 0


In [ ]:
# dumb but we'll update the function for outputs.shape

from tqdm import tqdm

def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, device):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for batch in progress_bar:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            # FLATTTEN AND RESHAPE
            outputs = outputs.view(-1, vocab_size)
            labels = labels.view(-1)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update progress bar
            avg_loss = total_loss / total
            accuracy = 100 * correct / total
            progress_bar.set_postfix(loss=avg_loss, accuracy=f'{accuracy:.2f}%')

        train_losses.append(total_loss / len(train_loader))
        train_accuracies.append(100 * correct / total)

        # Validation phase
        model.eval()
        total_loss = 0
        correct = 0
        total = 0
        progress_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Validation]')
        with torch.no_grad():
            for batch in progress_bar:
                inputs, labels = batch
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)

                outputs = outputs.view(-1, vocab_size)
                labels = labels.view(-1)

                loss = criterion(outputs, labels)

                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                # Update progress bar
                avg_loss = total_loss / total
                accuracy = 100 * correct / total
                progress_bar.set_postfix(loss=avg_loss, accuracy=f'{accuracy:.2f}%')

        val_losses.append(total_loss / len(val_loader))
        val_accuracies.append(100 * correct / total)

    return train_losses, val_losses, train_accuracies, val_accuracies

In [ ]:
torch.mps.empty_cache()


model = LLMModel4(vocab_size, n_hidden)
model.to(mps_device)  # Move model to MPS device

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
num_epochs = 10

train_losses, val_losses, train_accuracies, val_accuracies = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, device)

### multi-layer RNNs

### detour of detach

In [ ]:
import torch

# Creating a simple computation graph
x = torch.tensor([2.], requires_grad=True)
y = x * 2

# Without detach
z = y * y  # z = (x * 2) * (x * 2) = 4x^2
z.backward()  # Computes gradients for the whole graph
gradients_without_detach = x.grad.item()  # Should be dz/dx = 8x, which is 16 when x = 2

# Resetting gradients
x.grad.data.zero_()

# With detach
y_detached = y.detach()  # y_detached is a new tensor with the same value as y but no history of operations
# No need to call z.backward() because y_detached has no grad_fn
# and thus won't contribute to gradients in x

# The gradient of x should still be zero since the detached part does not contribute to the computation
gradients_with_detach = x.grad.item()

gradients_without_detach, gradients_with_detach


In [ ]:
## torch.stack()
lst = [torch.rand(2, 3) for i in range(3)]
print([x.shape for x in lst])

# stacking on dimension 1 maintains batch dimension, second dimension (1) becomes the timesteps
torch.stack(lst, dim=1).shape, torch.stack(lst, dim=1)

In [ ]:
torch.stack(lst, dim=0).shape, torch.stack(lst, dim=0)